In [37]:
import sys
print(sys.version)
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

2.7.10 (default, Feb  7 2017, 00:08:15) 
[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.34)]


In [38]:
from pyspark.mllib.regression import LabeledPoint
def transform_data(data):
    prediction_col = "failure"
    columns = []#["serial_number", "model", "capacity_bytes"]
    # Add Smart columns.
    for i in range(1, 255):
        test_col = "smart_" + str(i) + "_normalized"
        try:
            data[test_col]
            columns.append(test_col)
        except:
            pass
    return (float(data[prediction_col]), [0.0 if data[i] is None else data[i] for i in columns])

In [39]:
from pyspark.conf import *
spark = SparkSession.builder.master("local[1]").appName("SENG474-HD").getOrCreate()
df = spark.read.load('data_Q1_2017/2017-01-01.csv', 
                     format='com.databricks.spark.csv', 
                     header='true', 
                     inferSchema='true')

data = df.rdd.map(transform_data).collect()
labels, data = zip(*data)
print(data[0], type(data), labels[0], type(labels))

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:49329)
Traceback (most recent call last):
  File "/Library/Python/2.7/site-packages/py4j/java_gateway.py", line 990, in start
    self.socket.connect((self.address, self.port))
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:49329)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.3)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(data_train, labels_train)
predictions = clf.predict(data_test)
labels_and_predictions = zip(predictions, labels_test)
testErr = float(len(filter(lambda lp: lp[0] != lp[1], labels_and_predictions))) / float(len(data_test))
print("SKLean Test Error: {0:03f}".format(testErr))

In [ ]:
print(labels_and_predictions)